In [ ]:
## Import Libraries
import numpy as np
import gym
import random
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from collections import deque
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from IPython.display import clear_output

In [ ]:
## Initialize the CartPole environment
env = gym.make('CartPole-v1')

In [ ]:
## Set up DQN parameters
state_space_size = env.observation_space.shape[0]
action_space_size = env.action_space.n

In [ ]:
# Hyperparameters
num_episodes = 1000
max_steps_per_episode = 200
learning_rate = 0.001
discount_rate = 0.99
exploration_rate = 1
max_exploration_rate = 1
min_exploration_rate = 0.01
exploration_decay_rate = 0.001
batch_size = 64
memory_size = 10000
target_update_freq = 10

In [ ]:
# Define the Q-network
def build_model(state_space_size, action_space_size):
    model = Sequential()
    model.add(Dense(24, input_dim=state_space_size, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(action_space_size, activation='linear'))
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse')
    return model

In [ ]:
model = build_model(state_space_size, action_space_size)
target_model = build_model(state_space_size, action_space_size)
target_model.set_weights(model.get_weights())  # Initialize target model weights
memory = deque(maxlen=memory_size)
rewards_all_episodes = []

In [ ]:
## Training the DQN
def train_dqn():
    global exploration_rate
    global target_model

    for episode in range(num_episodes):
        state = env.reset()
        state = np.reshape(state, [1, state_space_size])
        done = False
        rewards_current_episode = 0

        for step in range(max_steps_per_episode):
            # Exploration-exploitation trade-off
            if np.random.rand() <= exploration_rate:
                action = env.action_space.sample()
            else:
                q_values = model.predict(state)
                action = np.argmax(q_values[0])
            
            next_state, reward, done, _ = env.step(action)
            next_state = np.reshape(next_state, [1, state_space_size])
            memory.append((state, action, reward, next_state, done))
            state = next_state
            rewards_current_episode += reward

            if len(memory) > batch_size:
                minibatch = random.sample(memory, batch_size)
                for state_b, action_b, reward_b, next_state_b, done_b in minibatch:
                    target = reward_b
                    if not done_b:
                        target += discount_rate * np.max(target_model.predict(next_state_b)[0])
                    target_f = model.predict(state_b)
                    target_f[0][action_b] = target
                    model.fit(state_b, target_f, epochs=1, verbose=0)
            
            if done:
                break

        if episode % target_update_freq == 0:
            target_model.set_weights(model.get_weights())  # Update target model

        exploration_rate = min_exploration_rate + \
            (max_exploration_rate - min_exploration_rate) * np.exp(-exploration_decay_rate*episode)
        
        rewards_all_episodes.append(rewards_current_episode)


In [ ]:
## Run training
train_dqn()

In [ ]:
## Calculate and print the average reward per thousand episodes
rewards_per_thousand_episodes = np.split(np.array(rewards_all_episodes), num_episodes/1000)
count = 1000

print("Average reward per thousand episodes")
for r in rewards_per_thousand_episodes:
    print(count, ": ", str(sum(r/1000)))
    count += 1000

In [ ]:
## Plotting the results
plt.figure(figsize=(12, 6))
plt.plot(range(num_episodes), rewards_all_episodes)
plt.xlabel('Episode')
plt.ylabel('Reward')
plt.title('Reward vs Episode')
plt.show()

In [ ]:
## Plot Q-values as a heatmap for a given state
def plot_q_values_heatmap_for_state(model, state):
    q_values = model.predict(state)[0]
    
    plt.figure(figsize=(8, 8))
    sns.heatmap(np.reshape(q_values, (1, -1)), cmap='viridis', annot=True, fmt=".1f", cbar=True)
    plt.title('Q-values for Given State')
    plt.xlabel('Action')
    plt.ylabel('Q-value')
    plt.show()

In [ ]:
## Visualize the agent's performance
def visualize_agent_performance(env, model):
    state = env.reset()
    state = np.reshape(state, [1, state_space_size])
    done = False
    total_reward = 0
    path = []

    for _ in range(max_steps_per_episode):
        q_values = model.predict(state)
        action = np.argmax(q_values[0])
        next_state, reward, done, _ = env.step(action)
        next_state = np.reshape(next_state, [1, state_space_size])
        total_reward += reward
        state = next_state
        path.append((state, action))

        if done:
            break

    print(f"Total Reward: {total_reward}")

    # Plot the path
    plt.figure(figsize=(12, 6))
    plt.plot([s[0][0] for s in path], label='Cart Position')
    plt.xlabel('Time Steps')
    plt.ylabel('Position')
    plt.title('Agent\'s Cart Position Over Time')
    plt.legend()
    plt.show()

In [ ]:
## Run the visualization
visualize_agent_performance(env, model)